<div align="center"> <h3><font color='cyan'>CSE 368: Introduction to Artificial Intelligence, FALL 2022 </font> 
<h1> Uniformed and Informed Search

<h3>Checkpoint: September 29, Thu, 11:59pm 
<h2><font color='red'>Final Due Date: </font> October 13, Thu, 11:59pm </div>


## 1. Problem Environment Definition

In [6]:
''' 
Nils Napp
Sliding Problem for AI-Class
'''

from queue import PriorityQueue
import random
import time



class State:
    """ State of sliding number puzzle
        Contains array of values called 'board' to indicate
        tile positions, and the position of tile '0', which
        indicates the empty space on the board.         """
    
    boardSize = 3

    def __init__(self, s = None):

        if s == None:
            
            # tiles is an iterator holding numbers 0 to 9
            tiles = range(self.boardSize * self.boardSize).__iter__()
            
            # below line reads numbers 0-8 stored in tiles and store them in 2d array (list of lists)
            self.board = [[next(tiles) for i in range(self.boardSize)] for j in range(self.boardSize)]

            #keep track of empty position
            self.position = [0,0]
        
        else:
            #copy the board
            self.board = []
            for row in s.board:
                self.board.append(list(row))

            #copy the positions    
            self.position = list(s.position)
            
    # converts to readable string to print
    def __str__(self):
        rstr = ''
        for row in self.board:
            rstr += str(row) + '\n'
        return rstr
    
    #overload to allow comparison of lists and states with ==
    def __eq__(self, other):
        if isinstance(other, State):
            return self.board == other.board
        elif isinstance(other, list):
            return self.board == other
        else:
            return NotImplemented

    #turn into immutable ojbect for set lookup
    def toTuple(self):
        tpl = ()
        for row in self.board:
            tpl += (tuple(row),)
        return tpl
    
    #create board from a list or tuple 
    def setBoard(self, brd):
        self.board = brd
        for row in range(self.boardSize):
            for col in range(self.boardSize):
                if self.board[row][col] == 0:
                    self.position = [row, col]
                    return None
        raise StandardError('Set board configuration does not have an empty spot!')

## 2. Defining Nodes

In [7]:
class Node:
    
    nodeCount = 0
    
    def __init__(self, p, a, c, s, d):
        
        #keep track of how many nodes were created
        self.__class__.nodeCount += 1    
        self.nodeID = self.nodeCount
        
        self.parent = p
        self.cost = c 
        self.action = a
        self.state = s
        self.depth = d

    def __lt__(self, other):
        # self < other
        return self.depth + calculate_h(self.state.toTuple()) < other.depth + calculate_h(other.state.toTuple())

    def __str__(self):
        rstr = 'NodeID: ' + str(self.nodeID) + '\n'
        if self.parent != None:
            rstr += 'Parent: ' + str(self.parent.nodeID) + '\n'
        if self.action != None:
            rstr += 'Action: ' + self.action  + '\n'
        rstr += 'Cost:   ' + str(self.cost) + '\n'
        rstr += 'State:\n' + str(self.state)
        return rstr
            
# creates and returns a new node which would be child of current node n being passed to the function    
def childNode(n, action, problem, d):
    return Node(n,action, n.cost + 1, problem.apply(action,State(n.state)), d)

goal_position = {}
idx = 0
for i in range(3):
    for j in range(3):
        goal_position[(i, j)] = idx
        idx += 1

def calculate_h(state_tuple):
    # arg should be passed as state.toTuple()
    # goal_position = { (0, 0): 0, (0, 1): 1, (0, 2): 2, (1, 0): 3 ... (3, 3): 8 }
    # calculating H by how many numbers are not in the right spot
    # calculate G = number of nodes moved from initial state (generally going to be +1 for each successor node)
    h = 0
    for i in range(3):
        for j in range(3):
            if state_tuple[i][j] != goal_position[(i, j)]:
                h += 1
    return h


## 3. Problem Definition

Possible actions of agent:
* 'U' - Up
* 'L' - Left
* 'D' - Down
* 'R' - Right

In [8]:
class Problem:
    """Class that defines a search problem"""

    def __init__(self):
        self.actions = ['U','L','D','R']
        self.initialState = 0
        self.goalState = 0

    def apply(self, a, s):

        #positions after move, still refers to s.position object
        post = s.position

        #make a copy
        pre = list(post)
        
        #compute post position
        if a == 'U':
            post[0] = max(pre[0] - 1, 0)
        elif a == 'L':            
            post[1] = max(pre[1] - 1, 0)
        elif a == 'D':
            post[0] = min(pre[0] + 1, s.boardSize - 1)
        elif a == 'R':
            post[1] = min(pre[1] + 1, s.boardSize - 1)
        else:
            print('Undefined action: ' + str(a))
            raise StandardError('Action not defined for this problem!')

        #store the old tile to slide/swap the tiles
        tile = s.board[pre[0]][pre[1]]
        
        s.board[pre[0]][pre[1]] = s.board[post[0]][post[1]]
        s.board[post[0]][post[1]] = tile      

#       print (pre, ' ', post,' ',s.board[pre[0]][pre[1]] , '<--', s.board[post[0]][post[1]])      

        return s
        
    def applicable(self, s):
        actionList = []

        #check if actions are applicable
        #Not in top row
        if s.position[0] > 0:
            actionList.append('U')

        #not in left most col
        if s.position[1] > 0:
            actionList.append('L')

        #not in bottom row
        if s.position[0] < (s.boardSize - 1):
            actionList.append('D')

        #not in right col
        if s.position[1] < (s.boardSize - 1):
            actionList.append('R')

        return actionList

    #test if currect state is goal state or not
    def goalTest(self, s):
        return self.goalState == s    

def applyRndMoves(numMoves, s, p):
    for i in range(numMoves):
        p.apply(p.actions[random.randint(0,3)], s)
    
def solution(node):
    ''' Returns actionList, cost of the solution generated from the node'''

    actions = []
    cost = node.cost

    while node.parent != None:
        actions.insert(0,node.action)
        node = node.parent    

    return actions, cost        

# Solving Sliding Problem
1.   Breadth First Search (BFS) Approach
> *It will find the closest solution, by searching layer-by-layer.*

> The BFS explores an entire layer before progressing.  Each layer consists of a list of nodes.  These lists are looped through, and expanded into a new list based on the available directions to explore for that node at it's given state.  Once a list is exhausted, it is replaced by the new list, and is then expanded.  The cycle ends once 15 layers, or 15 lists have been explored, or when a solution is found.  

2.   Depth First Search (DFS) Approach
> *It will find the first solution, by searching as deep as possible first.*

> The DFS method explores as deeply as possible, before branching out to adjacent nodes.  This was accomplished using recurrence and a for loop.  The for loop would loop through all directions in a function, and call on the function to explore downwards in each one.  Once the function reached it's maximum depth of 15 without finding a solution, it would return "None," allowing the previously halted function above it to explore a new, adjacent path, or return "None" itself to allow the function above it to explore a new path, etc.  

3.  Greedy Algorithm
> *The greedy algorithm works by choosing the best possible way in each step and then moving on to the next step until it reaches the end, without regard for the overall outcome. It picks the best immediate path, but does not consider the whole picture, so hence it is considered as greedy.*

4.  A* Approach
> *It will find the shortest distance between an initial and end point, by using an heuristic value which estimates the distance cost.*

> The A* method uses the formula f(n) = h(n) + g(n)
where, h(n) is the heuristic value which shows the actual cost from the current node to goal node.
g(n) tells us the current level.
f(n) is the actual cost path from the start node to the goal node.
It keeps track of all the visited nodes which helps in obtaining an optimal solution. Initially, the open list holds the Initial node. The next node chosen from the list is based on its g score, the node with the least g score is picked up and explored until we reach the goal state.



YOUR TASK:

Part I [40 points]:
1.   Implement and solve the problem using breadth-first search (BFS)
2.   Implement and solve the problem using depth-first search (DFS)

Part II [60 points]:

3.	Implement and solve the problem using Greedy Search Algorithm
4.	Implement and solve the problem using A* Algorithm

In [9]:
from collections import deque
class Searches:    

    def BFS(self, problem):
      #write your code here
        rootState = problem.initialState
        visited = set()
        #Initialize Root Node 
        rootNode = Node(None, None , 0, rootState, 0)
        qq = deque([rootNode])
        #continue until queue has no values in it 
        while qq:
            currentNode = qq.popleft()
            currentState = currentNode.state
            currentDepth = currentNode.depth
            if currentDepth > 15:
                #Returns None when all 15 the nodes from the 15th level have been exhausted 
                return None
            if problem.goalTest(currentState):
                return solution(currentNode)
            possibleDirections = problem.applicable(currentState)
            visited.add(currentState.toTuple())
            for direction in possibleDirections:
                temp = State(currentState)
                problem.apply(direction,temp)
                #Check if tuple version of state is in visited list
                if temp.toTuple() not in visited:
                    nextNode = childNode(currentNode,direction,problem,currentDepth + 1)
                    qq.append(nextNode)
        
                    
    def DFS(self, problem):
      #write your code here
        rootState = problem.initialState
        visited = set()
        #Initialize Root Node 
        rootNode = Node(None, None , 0, rootState, 0)
        stack = deque([rootNode])
        #continue until stack has nothing in it
        while stack:
            currentNode = stack.pop()
            currentState = currentNode.state
            currentDepth = currentNode.depth
            if currentDepth > 15:
                continue
            if problem.goalTest(currentState):
#                 print(currentNode)
                return solution(currentNode)
            possibleDirections = problem.applicable(currentState)
            visited.add(currentState.toTuple())
            for direction in possibleDirections:
                #Use the state the board came from
                temp = State(currentState)
                problem.apply(direction,temp)
                #Check if tuple version of state is in visited list
                if temp.toTuple() not in visited:
                    nextNode = childNode(currentNode,direction,problem,currentDepth + 1)
                    stack.append(nextNode)
        #Returns None when all 15 levels of decision tree has been exhausted
        return None        
                    

    def A_star(self, problem):
        rootState = problem.initialState
        visited = set()
        rootNode = Node(None, None , 0, rootState, 0)
        tupleInit = rootState.toTuple()
        cost = calculate_h(tupleInit)
        pq = PriorityQueue()
        pq.put((cost,rootNode))
        while pq:
            currentTuple = pq.get()
            currentNode = currentTuple[1]
            currentState = currentNode.state
            currentDepth = currentNode.depth
            if problem.goalTest(currentState):
#                 print(currentNode)
                return solution(currentNode)
            possibleDirections = problem.applicable(currentState)
            visited.add(currentState.toTuple())
            for direction in possibleDirections:
                temp = State(currentState)
                problem.apply(direction,temp)
                if temp.toTuple() not in visited and currentDepth + 1 <= 15:
                    nextNode = childNode(currentNode,direction,problem,currentDepth + 1)
                    nextCost = calculate_h(temp.toTuple())
                    pq.put((nextCost + currentNode.cost + 1, nextNode))
        return None
        

    def greedy(self, problem):
        rootState = problem.initialState
        visited = set()
        rootNode = Node(None, None , 0, rootState, 0)
        tupleInit = rootState.toTuple()
        cost = calculate_h(tupleInit)
        pq = PriorityQueue()
        pq.put((cost,rootNode))
        while pq:
            currentTuple = pq.get()
            currentNode = currentTuple[1]
            currentState = currentNode.state
            currentDepth = currentNode.depth
            if problem.goalTest(currentState):
#                 print(currentNode)
                return solution(currentNode)
            possibleDirections = problem.applicable(currentState)
            visited.add(currentState.toTuple())
            for direction in possibleDirections:
                temp = State(currentState)
                problem.apply(direction,temp)
                if temp.toTuple() not in visited and currentDepth + 1 <= 15:
                    nextNode = childNode(currentNode,direction,problem,currentDepth + 1)
                    nextCost = calculate_h(temp.toTuple())
                    pq.put((nextCost, nextNode))
        return None

# TEST YOUR SOLUTION 

In [11]:
if __name__ == '__main__':
       
    search = Searches()
    
    p = Problem()
    s = State()
    
    p.goalState = State(s)
    
    p.apply('R', s)
    p.apply('R', s)
    p.apply('D', s)
    p.apply('D', s)
    p.apply('L', s)
    
    p.initialState = State(s)
    
    print("Initial State\n", p.initialState)
    
    
### Uncommnet for testing your solution

    print('=== Running BFS ===')
    startTime = time.process_time()
    res=search.BFS(p)
    print(res)
    print("Time " + str(time.process_time()- startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0
 
    print('=== Running DFS ===')
    startTime = time.process_time()
    res = search.DFS(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

    print('=== Running Greedy ===')
    startTime = time.process_time()
    res = search.greedy(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

    print('=== Running A* ===')
    startTime = time.process_time()
    res = search.A_star(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0
    
    print("Generating Random Position\n")
    si = State(s)
    applyRndMoves(30,si,p)
    p.initialState = si
    print(si)
    
    print('=== Running BFS ===')
    startTime = time.process_time()
    res=search.BFS(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

    print('=== Runnning DFS ===')
    startTime = time.process_time()
    res = search.DFS(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

    print('=== Running Greedy ===')
    startTime = time.process_time()
    res = search.greedy(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

    print('=== Running A* ===')
    startTime = time.process_time()
    res = search.A_star(p)
    print(res)
    print("Time " + str(time.process_time() - startTime))
    print("Explored Nodes: " + str(Node.nodeCount) + "\n")
    Node.nodeCount = 0

Initial State
 [1, 2, 5]
[3, 4, 8]
[6, 0, 7]

=== Running BFS ===
(['R', 'U', 'U', 'L', 'L'], 5)
Time 0.0010850000000000026
Explored Nodes: 6737

=== Running DFS ===
(['R', 'U', 'U', 'L', 'L'], 5)
Time 0.013722000000000012
Explored Nodes: 1671

=== Running Greedy ===
(['R', 'U', 'U', 'L', 'L'], 5)
Time 0.0002590000000000092
Explored Nodes: 10

=== Running A* ===
(['R', 'U', 'U', 'L', 'L'], 5)
Time 0.0002670000000000172
Explored Nodes: 10

Generating Random Position

[1, 2, 5]
[6, 3, 4]
[0, 7, 8]

=== Running BFS ===
(['U', 'R', 'R', 'U', 'L', 'L'], 6)
Time 0.0009430000000001382
Explored Nodes: 116

=== Runnning DFS ===
(['U', 'R', 'R', 'U', 'L', 'L'], 6)
Time 0.018758000000000052
Explored Nodes: 2553

=== Running Greedy ===
(['U', 'R', 'R', 'U', 'L', 'L'], 6)
Time 0.0002329999999999277
Explored Nodes: 14

=== Running A* ===
(['U', 'R', 'R', 'U', 'L', 'L'], 6)
Time 0.00021399999999993646
Explored Nodes: 14

